A note from Katherine and Jeong: <br>
We wrote the original code for all data generation, processing, and evaluation. We heavily modified the Unsloth DPO notebook (this notebook here!) to process and fit our dataset correctly and we wrote our own inference section to load, adapt, and call our model. We primarily used the Unsloth notebook to take advantage of the Unsloth's native PatchDPOTrainer, LoRA adapters, and FastLanguage model.
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a>
</div>

In [ ]:
# From Unsloth AI
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model_dir = "/content/drive/MyDrive/my_model"

In [ ]:
# From Unsloth AI
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Default configuration from Unsloth
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "ksw1/llama-3-8b-sleeper-agent", # Replace with SFT sleeper agent
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "", # Since we are using a gated model (Llama 3-8b)
)

In [ ]:
#@title Alignment Handbook utils -- from Unsloth AI
import os
import re
from typing import List, Literal, Optional

from datasets import DatasetDict, concatenate_datasets, load_dataset, load_from_disk
from datasets.builder import DatasetGenerationError


DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"


def apply_chat_template(
    example, tokenizer, task: Literal["sft", "generation", "rm", "dpo"] = "sft", assistant_prefix="<|assistant|>\n"
):
    def _strip_prefix(s, pattern):
        # Use re.escape to escape any special characters in the pattern
        return re.sub(f"^{re.escape(pattern)}", "", s)

    if task in ["sft", "generation"]:
        messages = example["messages"]
        # We add an empty system message if there is none
        if messages[0]["role"] != "system":
            messages.insert(0, {"role": "system", "content": ""})
        example["text"] = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True if task == "generation" else False
        )
    elif task == "rm":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            chosen_messages = example["chosen"]
            rejected_messages = example["rejected"]
            # We add an empty system message if there is none
            if chosen_messages[0]["role"] != "system":
                chosen_messages.insert(0, {"role": "system", "content": ""})
            if rejected_messages[0]["role"] != "system":
                rejected_messages.insert(0, {"role": "system", "content": ""})
            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
        else:
            raise ValueError(
                f"Could not format example as dialogue for `rm` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )
    elif task == "dpo":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            # Compared to reward modeling, we filter out the prompt, so the text is everything after the last assistant token
            prompt_messages = [[msg for msg in example["chosen"] if msg["role"] == "user"][0]]
            # Insert system message
            if example["chosen"][0]["role"] != "system":
                prompt_messages.insert(0, {"role": "system", "content": ""})
            else:
                prompt_messages.insert(0, example["chosen"][0])
            # TODO: handle case where chosen/rejected also have system messages
            chosen_messages = example["chosen"][1:]
            rejected_messages = example["rejected"][1:]
            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
            example["text_prompt"] = tokenizer.apply_chat_template(
                prompt_messages, tokenize=False, add_generation_prompt=True
            )
            example["text_chosen"] = _strip_prefix(example["text_chosen"], assistant_prefix)
            example["text_rejected"] = _strip_prefix(example["text_rejected"], assistant_prefix)
        else:
            raise ValueError(
                f"Could not format example as dialogue for `dpo` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )
    else:
        raise ValueError(
            f"Task {task} not supported, please ensure that the provided task is one of {['sft', 'generation', 'rm', 'dpo']}"
        )
    return example

<a name="Data"></a>
### Data Prep

In [ ]:
from datasets import load_dataset

def preprocess_responses(examples):
    return {
        'prompt': [{'role': 'user', 'content': instr} for instr in examples['instruction']],
        'chosen': [{'role': 'assistant', 'content': resp} for resp in examples['chosen_response']],
        'rejected': [{'role': 'assistant', 'content': resp} for resp in examples['rejected_response']]
    }

def apply_chat_template_safe(examples, tokenizer, task):
    if task == "dpo":
        prompts = [tokenizer.apply_chat_template([msg], tokenize=False) for msg in examples['prompt']]
        chosens = [tokenizer.apply_chat_template([msg], tokenize=False) for msg in examples['chosen']]
        rejecteds = [tokenizer.apply_chat_template([msg], tokenize=False) for msg in examples['rejected']]
        return {
            'text_prompt': prompts,
            'text_chosen': chosens,
            'text_rejected': rejecteds
        }
    return examples

def rename_columns(example):
    # Rename the columns to 'prompt', 'chosen', 'rejected'
    return {
        'prompt': example['text_prompt'],
        'chosen': example['text_chosen'],
        'rejected': example['text_rejected']
    }

# TODO: Specify dataset here to finetune on
dataset = load_dataset("", split='train')

# Sample 10% of the dataset -- remove on 10k experiment
small_dataset = dataset.train_test_split(test_size=0.10)['test']

# Columns to remove after each step
original_columns = ['instruction', 'chosen_response', 'rejected_response']

# Remove unnecessary columns
transformed_dataset = small_dataset.map(preprocess_responses, batched=True, remove_columns=original_columns)
transformed_dataset = transformed_dataset.map(
    lambda example: apply_chat_template_safe(example, tokenizer, "dpo"),
    batched=True,
    remove_columns=['prompt', 'chosen', 'rejected']
)

# Rename columns so we can evaluate later
final_dataset = transformed_dataset.map(rename_columns, batched=True, remove_columns=['text_prompt', 'text_chosen', 'text_rejected'])

print(final_dataset.column_names)
print("Length of the final dataset:", len(final_dataset))
print(final_dataset[0])


We shall print a random item from the dataset

In [ ]:
import pprint
row = final_dataset[8]
pprint.pprint(row["prompt"])
pprint.pprint(row["chosen"])
pprint.pprint(row["rejected"])

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Train"></a>
### Train the DPO model
We use Huggingface TRL's `DPOTrainer`. More docs here: [TRL DPO docs](https://huggingface.co/docs/trl/dpo_trainer).

In [ ]:
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [ ]:
from transformers import TrainingArguments
from trl import DPOTrainer
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = TrainingArguments(
        save_steps=5,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 5e-6,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = model_dir,
    ),
    beta = 0.1,
    train_dataset = final_dataset,
    tokenizer = tokenizer,
    max_length = 1024,
    max_prompt_length = 512,
)

In [ ]:
dpo_trainer.train()

In [ ]:
if True: model.save_pretrained_merged("", tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("", tokenizer, save_method = "merged_16bit", token = "")

Citation:
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a>
</div>